# Use O2C_Template Customized Notebook Template

In [1]:
import pandas as pd
import numpy as np

In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [3]:
table_name = 'ORDER_TO_CASH_ENRICHED'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [4]:
df = df[df['INVOICESTATUS'] == 'Paid']
df_new = df[df['INVOICESTATUS'] != 'Paid']

In [5]:
df.shape, df_new.shape

((170290, 37), (0, 37))

In [6]:
# Assuming df is your dataframe
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])
df['DELIVERYDATE'] = pd.to_datetime(df['DELIVERYDATE'])
df['INVOICEDATE'] = pd.to_datetime(df['INVOICEDATE'])
df['PAYMENTDATE'] = pd.to_datetime(df['PAYMENTDATE'])
df['INVOICEDUEDATE'] = pd.to_datetime(df['INVOICEDUEDATE'])
df['DELIVEREDON'] = pd.to_datetime(df['DELIVEREDON'])

In [7]:
# Total Sales Volume
total_sales_volume = df.groupby('PRODUCTID')['ORDERQUANTITY'].sum().reset_index(name='Total Sales Volume')

In [8]:
# Total Sales Value
total_sales_value = df.groupby('PRODUCTID')['ORDERVALUE'].sum().reset_index(name='Total Sales Value')

In [9]:
# Average Order Quantity
average_order_quantity = df.groupby('PRODUCTID')['ORDERQUANTITY'].mean().reset_index(name='Average Order Quantity')

In [10]:
# Average Unit Price
average_unit_price = df.groupby('PRODUCTID')['UNITPRICE'].mean().reset_index(name='Average Unit Price')

In [11]:
# Number of Orders
number_of_orders = df.groupby('PRODUCTID')['ORDERID'].nunique().reset_index(name='Number of Orders')

In [66]:
df.head()

,CUSTOMERID,CUSTOMERNAME,CREDITRATING,CUSTOMERTYPE,COMPANYTYPE,CONTACTDETAILS,EMAILDETAILS,ADDRESSDETAILS,ADMINDETAILS,CREDITLIMIT,...,PREFERRED_PRODUCT_TYPE,AVG_ORDER_PROCESSING_TIME,AVG_INVOICE_PROCESSING_TIME,AVG_DELIVERY_DELAY,AVG_PAYMENT_DELAY,TOTAL_DELAYS,ORDER_CONSISTENCY,INVOICE_CONSISTENCY,DELIVERY_CONSISTENCY,PAYMENT_CONSISTENCY
0,CID-4a14e75e-cd20-44a1-9696-d5eb1746b894,Anabrige LLP,fair,Construction Materials,and Sons,3788905298,osbornecorey@merritt-patrick.com,"5779 Michael Meadow\nSouth Connorchester, VA 9...",New Jersey,59238,...,PVC Pipes,6.958333,8.415833,-1.441667,2.423333,1178,2.012454,4.301163,1.139889,4.363675
3,CID-f922f515-1a7a-4640-959a-3c2f093f4d47,FULCRO,very poor,Automotive,and Sons,860-913-5769x082,richard89@bryan-ayala.net,"650 Peter Forks\nSouth Tonyborough, FL 20143",New York,99235,...,Radiator Coolant,7.079389,7.890840,-1.519084,1.933588,543,2.009478,4.315604,1.094127,4.457354
4,CID-169eb6d1-27a1-4859-b7c8-5073ca1eb201,Taylor Ltd,good,Industrial Equipment,LLC,050.237.0903x440,kenneth06@jones.com,"562 Melanie Curve Apt. 149\nEast Kathleen, ID ...",South Carolina,53969,...,Industrial Robot,7.130435,8.052003,-1.472293,2.023870,647,2.027760,4.310261,1.165616,4.459079
6,CID-03ffafe5-c975-4691-9124-d7c65cdcef54,PrimeSource,good,Office Supplies,Inc,+1-029-747-3627,vazqueztracy@cole.com,"35794 Miller River Apt. 153\nEast Bruce, MO 93432",Arizona,23701,...,Office Stationery Set,7.017102,8.324032,-1.601260,2.335734,816,2.076613,4.180322,1.111106,4.246013
7,CID-e055738a-560d-469c-9e7e-2c27a25177b5,Visionary Solutions,very good,Food Beverages,Inc,705-417-3307x5777,paullindsey@parker-kelly.com,"21856 Green Circles\nMarymouth, NV 59842",Wyoming,91121,...,Gourmet Chocolate,6.931507,7.993151,-1.501712,1.994007,575,2.057969,4.330987,1.109280,4.394333


In [67]:
df_train_sf=my_session.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())
df_train_sf.write.mode("overwrite").save_as_table("FDC_HORIZONTAL.O2C_GOLD.ORDER_TO_CASH_PRODUCT_KPI")